In [74]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import os
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import re

import xlrd
import holidays

import json, requests
from geopy.geocoders import Nominatim


from zeep import Client
import warnings

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Скачать нерегулируемые цены с АТС

In [230]:
outfolder = 'DATA/ATS_unreg_prices/'

prefs = {
    "download.default_directory": os.path.abspath(outfolder),
    "download.directory_upgrade": True,
    "download.prompt_for_download": False,
}

chromeOptions = ChromeOptions()
chromeOptions.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chromeOptions)

In [102]:
dt0   = datetime(2014,1,1,0,0)
dtend = datetime(2022,9,1,0,0)

dt = dt0

base_url = 'https://www.atsenergo.ru/dload/retail'

while(dt < dtend):
    time.sleep(2)
    
    d1 = str(dt.year) + str(dt.month).zfill(2) + str(dt.day).zfill(2)
    d2 = str(dt.year) + str(dt.month+1).zfill(2) + str(dt.day+9).zfill(2)
    d3 = str(dt.month).zfill(2) + str(dt.year)
    URL = f'{base_url}/{d1}/{d2}_KUZBSBYT_PKUZBENE_{d3}_gtp_1st_stage.xls'
    driver.get(URL)

    dt = dt + relativedelta(months=1)

In [231]:
dt_list = [datetime(2014,12,1,0,0),
           datetime(2015,12,1,0,0),
           datetime(2016,12,1,0,0),
           datetime(2017,12,1,0,0),
           datetime(2018,12,1,0,0),
           datetime(2019,12,1,0,0),
           datetime(2020,12,1,0,0),
           datetime(2021,12,1,0,0),
           datetime(2022,12,1,0,0)]

base_url = 'https://www.atsenergo.ru/dload/retail'

for dt in dt_list:
    time.sleep(2)
    
    d1 = str(dt.year) + str(dt.month).zfill(2) + str(dt.day).zfill(2)
    d2 = str(dt.year+1) + str(1).zfill(2) + str(dt.day+9).zfill(2)
    d3 = str(dt.month).zfill(2) + str(dt.year)
    URL = f'{base_url}/{d1}/{d2}_KUZBSBYT_PKUZBENE_{d3}_gtp_1st_stage.xls'
    driver.get(URL)

    dt = dt + relativedelta(years=1)

In [4]:
def parse_data(file_name, path):
    dt = datetime(int(re.findall(r'\d+', file_name)[1][2:]), int(re.findall(r'\d+', file_name)[1][:2]), 1, 0, 0)
    if dt >= datetime(2021,3,1,0,0):
        skiprows = 47
    else:
        skiprows = 39
    df = pd.read_excel(path + '\\' + file_name,
                       decimal=',',
                       skiprows=skiprows,
                       names=['date', 'hour', 'x1', 'x2', 'x3', 'price']).dropna()
    df = df.drop(df[df.hour == 24].index)
    df.index = pd.to_datetime(df["date"].map(str) + ' ' + df["hour"].map(int).map(str) + ':00:00', format="%d.%m.%Y %H:%M:%S")
    return df[['price']]

In [5]:
outfolder = 'DATA/ATS_unreg_prices/'
path = os.path.abspath(outfolder)
ats = pd.DataFrame()

for file in os.listdir(os.fsencode(path)):
    ats = pd.concat([ats, parse_data(os.fsdecode(file), path)])
ats.index.name = 'timestamp'
ats = ats.sort_index()
ats['price'] = pd.to_numeric(ats['price'], errors='coerce').fillna(0)

ats.to_pickle("DATASET/ats.pkl")

ats

,price
timestamp,
2014-01-01 00:00:00,729.11
2014-01-01 01:00:00,716.33
2014-01-01 02:00:00,709.10
2014-01-01 03:00:00,702.10
2014-01-01 04:00:00,710.92
...,...
2022-08-31 19:00:00,1358.39
2022-08-31 20:00:00,1199.29
2022-08-31 21:00:00,1097.42


## Скачать индексы равновесных цен на РСВ по Кем. области

In [9]:
outfolder = 'DATA/ATS_node_prices_report/'

prefs = {
    "download.default_directory": os.path.abspath(outfolder),
    "download.directory_upgrade": True,
    "download.prompt_for_download": False,
}

chromeOptions = ChromeOptions()
chromeOptions.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chromeOptions)

In [12]:
dt0 = datetime(2020,9,15,0,0)
dtend = datetime(2022,9,14,0,0)

dt = dt0

while(dt < dtend):   
    d1 = str(dt.year) + str(dt.month).zfill(2) + str(dt.day).zfill(2)
    URL = f'https://www.atsenergo.ru/nreport?rname=big_nodes_prices_pub&rdate={d1}&region=sib'

    driver.get(URL)

    link = driver.find_element(By.LINK_TEXT, f'{d1}_sib_big_nodes_prices_pub.xls')
    link.click()
    
    print(f'{dt} downloaded...')
    
    dt = dt + timedelta(days=1)
    time.sleep(2)

2020-09-15 00:00:00 downloaded...
2020-09-16 00:00:00 downloaded...
2020-09-17 00:00:00 downloaded...
2020-09-18 00:00:00 downloaded...
2020-09-19 00:00:00 downloaded...
2020-09-20 00:00:00 downloaded...
2020-09-21 00:00:00 downloaded...
2020-09-22 00:00:00 downloaded...
2020-09-23 00:00:00 downloaded...
2020-09-24 00:00:00 downloaded...
2020-09-25 00:00:00 downloaded...
2020-09-26 00:00:00 downloaded...
2020-09-27 00:00:00 downloaded...
2020-09-28 00:00:00 downloaded...
2020-09-29 00:00:00 downloaded...
2020-09-30 00:00:00 downloaded...
2020-10-01 00:00:00 downloaded...
2020-10-02 00:00:00 downloaded...
2020-10-03 00:00:00 downloaded...
2020-10-04 00:00:00 downloaded...
2020-10-05 00:00:00 downloaded...
2020-10-06 00:00:00 downloaded...
2020-10-07 00:00:00 downloaded...
2020-10-08 00:00:00 downloaded...
2020-10-09 00:00:00 downloaded...
2020-10-10 00:00:00 downloaded...
2020-10-11 00:00:00 downloaded...
2020-10-12 00:00:00 downloaded...
2020-10-13 00:00:00 downloaded...
2020-10-14 00:

2021-05-14 00:00:00 downloaded...
2021-05-15 00:00:00 downloaded...
2021-05-16 00:00:00 downloaded...
2021-05-17 00:00:00 downloaded...
2021-05-18 00:00:00 downloaded...
2021-05-19 00:00:00 downloaded...
2021-05-20 00:00:00 downloaded...
2021-05-21 00:00:00 downloaded...
2021-05-22 00:00:00 downloaded...
2021-05-23 00:00:00 downloaded...
2021-05-24 00:00:00 downloaded...
2021-05-25 00:00:00 downloaded...
2021-05-26 00:00:00 downloaded...
2021-05-27 00:00:00 downloaded...
2021-05-28 00:00:00 downloaded...
2021-05-29 00:00:00 downloaded...
2021-05-30 00:00:00 downloaded...
2021-05-31 00:00:00 downloaded...
2021-06-01 00:00:00 downloaded...
2021-06-02 00:00:00 downloaded...
2021-06-03 00:00:00 downloaded...
2021-06-04 00:00:00 downloaded...
2021-06-05 00:00:00 downloaded...
2021-06-06 00:00:00 downloaded...
2021-06-07 00:00:00 downloaded...
2021-06-08 00:00:00 downloaded...
2021-06-09 00:00:00 downloaded...
2021-06-10 00:00:00 downloaded...
2021-06-11 00:00:00 downloaded...
2021-06-12 00:

2022-01-10 00:00:00 downloaded...
2022-01-11 00:00:00 downloaded...
2022-01-12 00:00:00 downloaded...
2022-01-13 00:00:00 downloaded...
2022-01-14 00:00:00 downloaded...
2022-01-15 00:00:00 downloaded...
2022-01-16 00:00:00 downloaded...
2022-01-17 00:00:00 downloaded...
2022-01-18 00:00:00 downloaded...
2022-01-19 00:00:00 downloaded...
2022-01-20 00:00:00 downloaded...
2022-01-21 00:00:00 downloaded...
2022-01-22 00:00:00 downloaded...
2022-01-23 00:00:00 downloaded...
2022-01-24 00:00:00 downloaded...
2022-01-25 00:00:00 downloaded...
2022-01-26 00:00:00 downloaded...
2022-01-27 00:00:00 downloaded...
2022-01-28 00:00:00 downloaded...
2022-01-29 00:00:00 downloaded...
2022-01-30 00:00:00 downloaded...
2022-01-31 00:00:00 downloaded...
2022-02-01 00:00:00 downloaded...
2022-02-02 00:00:00 downloaded...
2022-02-03 00:00:00 downloaded...
2022-02-04 00:00:00 downloaded...
2022-02-05 00:00:00 downloaded...
2022-02-06 00:00:00 downloaded...
2022-02-07 00:00:00 downloaded...
2022-02-08 00:

2022-09-08 00:00:00 downloaded...
2022-09-09 00:00:00 downloaded...
2022-09-10 00:00:00 downloaded...
2022-09-11 00:00:00 downloaded...
2022-09-12 00:00:00 downloaded...
2022-09-13 00:00:00 downloaded...


In [2]:
outfolder = 'DATA/ATS_node_prices_report/'
path = os.path.abspath(outfolder)
node_prices = pd.DataFrame()

dt_start = datetime(2020,8,2,0,0)

for file in os.listdir(os.fsencode(path)):
    date = '.'.join(re.findall(r'\d+', os.fsdecode(file)))
    date = datetime.strptime(date, '%Y%m%d')
    
    if date < dt_start:
        pass
    else:    
        resuts = {}
        for i in range(24):
            date = date.replace(hour=i)
            df = pd.read_excel(path + '\\' + os.fsdecode(file),
                           skiprows=2,
                           sheet_name=i,
                           usecols=['Субъект РФ', 'Цена, руб'])
            price = df[df['Субъект РФ'] == 'Кемеровская область']['Цена, руб'].mean()
            resuts.update({date : price})
        df = pd.DataFrame(pd.Series(resuts, name='price'))
        node_prices = pd.concat([node_prices, df])
        if date.day == 1:
            node_prices.to_pickle("DATA/ATS_node_prices_report/checkpoint_1.pkl")
        print(os.fsdecode(file))

20200802_sib_big_nodes_prices_pub.xls
20200803_sib_big_nodes_prices_pub.xls
20200804_sib_big_nodes_prices_pub.xls
20200805_sib_big_nodes_prices_pub.xls
20200806_sib_big_nodes_prices_pub.xls
20200807_sib_big_nodes_prices_pub.xls
20200808_sib_big_nodes_prices_pub.xls
20200809_sib_big_nodes_prices_pub.xls
20200810_sib_big_nodes_prices_pub.xls
20200811_sib_big_nodes_prices_pub.xls
20200812_sib_big_nodes_prices_pub.xls
20200813_sib_big_nodes_prices_pub.xls
20200814_sib_big_nodes_prices_pub.xls
20200815_sib_big_nodes_prices_pub.xls
20200816_sib_big_nodes_prices_pub.xls
20200817_sib_big_nodes_prices_pub.xls
20200818_sib_big_nodes_prices_pub.xls
20200819_sib_big_nodes_prices_pub.xls
20200820_sib_big_nodes_prices_pub.xls
20200821_sib_big_nodes_prices_pub.xls
20200822_sib_big_nodes_prices_pub.xls
20200823_sib_big_nodes_prices_pub.xls
20200824_sib_big_nodes_prices_pub.xls
20200825_sib_big_nodes_prices_pub.xls
20200826_sib_big_nodes_prices_pub.xls
20200827_sib_big_nodes_prices_pub.xls
20200828_sib

20210308_sib_big_nodes_prices_pub.xls
20210309_sib_big_nodes_prices_pub.xls
20210310_sib_big_nodes_prices_pub.xls
20210311_sib_big_nodes_prices_pub.xls
20210312_sib_big_nodes_prices_pub.xls
20210313_sib_big_nodes_prices_pub.xls
20210314_sib_big_nodes_prices_pub.xls
20210315_sib_big_nodes_prices_pub.xls
20210316_sib_big_nodes_prices_pub.xls
20210317_sib_big_nodes_prices_pub.xls
20210318_sib_big_nodes_prices_pub.xls
20210319_sib_big_nodes_prices_pub.xls
20210320_sib_big_nodes_prices_pub.xls
20210321_sib_big_nodes_prices_pub.xls
20210322_sib_big_nodes_prices_pub.xls
20210323_sib_big_nodes_prices_pub.xls
20210324_sib_big_nodes_prices_pub.xls
20210325_sib_big_nodes_prices_pub.xls
20210326_sib_big_nodes_prices_pub.xls
20210327_sib_big_nodes_prices_pub.xls
20210328_sib_big_nodes_prices_pub.xls
20210329_sib_big_nodes_prices_pub.xls
20210330_sib_big_nodes_prices_pub.xls
20210331_sib_big_nodes_prices_pub.xls
20210401_sib_big_nodes_prices_pub.xls
20210402_sib_big_nodes_prices_pub.xls
20210403_sib

20211010_sib_big_nodes_prices_pub.xls
20211011_sib_big_nodes_prices_pub.xls
20211012_sib_big_nodes_prices_pub.xls
20211013_sib_big_nodes_prices_pub.xls
20211014_sib_big_nodes_prices_pub.xls
20211015_sib_big_nodes_prices_pub.xls
20211016_sib_big_nodes_prices_pub.xls
20211017_sib_big_nodes_prices_pub.xls
20211018_sib_big_nodes_prices_pub.xls
20211019_sib_big_nodes_prices_pub.xls
20211020_sib_big_nodes_prices_pub.xls
20211021_sib_big_nodes_prices_pub.xls
20211022_sib_big_nodes_prices_pub.xls
20211023_sib_big_nodes_prices_pub.xls
20211024_sib_big_nodes_prices_pub.xls
20211025_sib_big_nodes_prices_pub.xls
20211026_sib_big_nodes_prices_pub.xls
20211027_sib_big_nodes_prices_pub.xls
20211028_sib_big_nodes_prices_pub.xls
20211029_sib_big_nodes_prices_pub.xls
20211030_sib_big_nodes_prices_pub.xls
20211031_sib_big_nodes_prices_pub.xls
20211101_sib_big_nodes_prices_pub.xls
20211102_sib_big_nodes_prices_pub.xls
20211103_sib_big_nodes_prices_pub.xls
20211104_sib_big_nodes_prices_pub.xls
20211105_sib

20220515_sib_big_nodes_prices_pub.xls
20220516_sib_big_nodes_prices_pub.xls
20220517_sib_big_nodes_prices_pub.xls
20220518_sib_big_nodes_prices_pub.xls
20220519_sib_big_nodes_prices_pub.xls
20220520_sib_big_nodes_prices_pub.xls
20220521_sib_big_nodes_prices_pub.xls
20220522_sib_big_nodes_prices_pub.xls
20220523_sib_big_nodes_prices_pub.xls
20220524_sib_big_nodes_prices_pub.xls
20220525_sib_big_nodes_prices_pub.xls
20220526_sib_big_nodes_prices_pub.xls
20220527_sib_big_nodes_prices_pub.xls
20220528_sib_big_nodes_prices_pub.xls
20220529_sib_big_nodes_prices_pub.xls
20220530_sib_big_nodes_prices_pub.xls
20220531_sib_big_nodes_prices_pub.xls
20220601_sib_big_nodes_prices_pub.xls
20220602_sib_big_nodes_prices_pub.xls
20220603_sib_big_nodes_prices_pub.xls
20220604_sib_big_nodes_prices_pub.xls
20220605_sib_big_nodes_prices_pub.xls
20220606_sib_big_nodes_prices_pub.xls
20220607_sib_big_nodes_prices_pub.xls
20220608_sib_big_nodes_prices_pub.xls
20220609_sib_big_nodes_prices_pub.xls
20220610_sib

ValueError: time data '' does not match format '%Y%m%d'

In [2]:
node_prices = pd.concat([pd.read_pickle("DATA/ATS_node_prices_report/checkpoint.pkl"),
                        pd.read_pickle("DATA/ATS_node_prices_report/checkpoint_1.pkl")])
node_prices.columns = ['node_price']

node_prices.to_pickle("DATA/ATS_node_prices_report/node_prices.pkl")

node_prices

,node_price
2019-09-15 00:00:00,596.798060
2019-09-15 01:00:00,596.328592
2019-09-15 02:00:00,594.732145
2019-09-15 03:00:00,590.206797
2019-09-15 04:00:00,721.414405
...,...
2022-09-01 19:00:00,1455.479326
2022-09-01 20:00:00,1339.870108
2022-09-01 21:00:00,1207.049875
2022-09-01 22:00:00,1115.709080


## Скачать плановое потребление и генерацию по Кем. области

In [ ]:
client = Client ('http://br.so-ups.ru:8091/PublicApi/PublicApiService.svc?wsdl')
regions = client.service.GetRegions()

def sobr_scraper(region_code=32, 
                 power_sys_ID=610000, 
                 dt0=datetime(2014,1,1,0,0),
                 dtend=datetime.now(), 
                 region_name=None, 
                 test=False):
    df = pd.DataFrame(columns=['datetime', 'plannedConsumption', 
                               'actualConsumption', 'plannedGeneration', 
                               'actualGeneration', 'IBRavg', 'IBRmax'])
    dt = dt0
    
    if region_name: 
        folder = region_name + '/'
    else:
        folder = None
        
    outfolder = 'DATA/SOBR_data/' + folder
    os.makedirs(outfolder, exist_ok=True)
    i = 0
    while(dt < dtend):
        
        time.sleep(1)
        
        if i % 24 == 0: print(dt)
        i += 1
        
        res = client.service.GetMapBalancingMarketSubregionsData(
            regionKey={'SubjectID': region_code, 'PowerSystemID': power_sys_ID}, 
            date=dt, 
            hour=dt.hour, 
            returnType='Json')
        jsondata = json.loads(res)
        
        try:
            jsondata = next(item for item in jsondata if item['SubjectID']==str(region_code))
            if test is not False: 
                print(jsondata)
                return None
        except:
            print('Неверно указан ID региона!')
            return None
        
        try:
            plannedConsumption = jsondata['EUsePlan']
            actualConsumption  = jsondata['EUseFact']
            plannedGeneration  = jsondata['GenPlan']
            actualGeneration   = jsondata['GenFact']
            IBRavg = jsondata['IBRavg']
            IBRmax = jsondata['IBRmax']

            df = df.append({'datetime' : dt, 
                            'plannedConsumption' : plannedConsumption,  
                            'actualConsumption' : actualConsumption, 
                            'plannedGeneration' : plannedGeneration, 
                            'actualGeneration' : actualGeneration,
                            'IBRavg' : IBRavg,
                            'IBRmax' : IBRmax
                           },
                           ignore_index=True)
    
        except:
            print('No data for this date:', dt)
            pass

        dt = dt + timedelta(hours=1)

        if dt.day == 1 and dt.hour == 0:
            old_dt = dt - timedelta(hours=1)
            print(f'Saved data: {old_dt.year}-{old_dt.month} ...')
            writer = pd.ExcelWriter(outfolder + region_name + f' {old_dt.year}-{old_dt.month}' + '.xlsx', engine='xlsxwriter')
            df.to_excel(writer, sheet_name='Sheet')
            writer.save()
            df = pd.DataFrame(columns=['datetime', 'plannedConsumption', 
                               'actualConsumption', 'plannedGeneration', 
                               'actualGeneration', 'IBRavg', 'IBRmax'])
            
    writer = pd.ExcelWriter(outfolder + region_name + '.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet')
    writer.save()

In [ ]:
sobr_scraper(region_code=32, power_sys_ID=610000, dt0=datetime(2017,12,1), dtend=datetime(2022,9,1), 
             region_name='Kemerovo', test=False)

In [60]:
def load_sobr(file_name, path):
    if file_name == 'Kemerovo 2014-1_2019-9.xlsx':
        df = pd.read_excel(path+file_name,
                           names=['no','date', 'load_planned_sobr', 'load_actual_sobr', 
                                  'generate_planned_sobr', 'generate_actual_sobr'])    
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df = df.drop(columns=['no'])
        return df
    else:
        df = pd.read_excel(path+file_name,
                   index_col='datetime')
        df.columns = ['no','load_planned_sobr', 'load_actual_sobr', 
                      'generate_planned_sobr', 'generate_actual_sobr', 'IBRavg', 'IBRmax']
        return df.drop(columns=['no', 'IBRavg', 'IBRmax'])

def get_sobr_data():
    path = os.getcwd() + f'\\DATA\\SOBR_data\\'
    sobr = pd.DataFrame(columns=['load_planned_sobr', 'load_actual_sobr', 
                                 'generate_planned_sobr', 'generate_actual_sobr'])
    
    for file in os.listdir(os.fsencode(path)):
        sobr = pd.concat([sobr, load_sobr(os.fsdecode(file), path)])
    sobr.index.name = 'timestamp'
    return sobr

In [62]:
sobr = get_sobr_data()
sobr.to_pickle("DATASET/sobr_data.pkl")
sobr

,load_planned_sobr,load_actual_sobr,generate_planned_sobr,generate_actual_sobr
timestamp,,,,
2014-01-01 00:00:00,3429.0,3498.0,2255.0,2248.0
2014-01-01 01:00:00,3462.0,3405.0,2255.0,2274.0
2014-01-01 02:00:00,3550.0,3458.0,2255.0,2275.0
2014-01-01 03:00:00,3539.0,3456.0,2255.0,2265.0
2014-01-01 04:00:00,3518.0,3407.0,2255.0,2255.0
...,...,...,...,...
2022-08-31 19:00:00,3069.159912,3007.795898,2032.906006,2072.411377
2022-08-31 20:00:00,2963.500977,2905.985352,2054.600098,2066.522949
2022-08-31 21:00:00,2950.384033,2904.853027,2059.600098,2033.241821


## Скачать погоду

In [ ]:
def get_weather_from_nasa_power(region_name, dt_start='20140101', dt_end='20220901'):
    geolocator = Nominatim(user_agent="whatshouldiwritehere")
    location = geolocator.geocode(region_name)
    locations = (location.latitude, location.longitude)

    params   = ['T2M']
    # https://power.larc.nasa.gov/#resources

    base_url = r"https://power.larc.nasa.gov/api/temporal/hourly/point?parameters={params}&community=RE&longitude={longitude}&latitude={latitude}&start={dt_start}&end={dt_end}&format=JSON"

    api_request_url = base_url.format(longitude=locations[1], 
                                      latitude=locations[0],
                                      dt_start=dt_start,
                                      dt_end=dt_end,
                                      params=','.join(params))

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    content = json.loads(response.content.decode('utf-8'))

    df = pd.DataFrame()
    try:
        results = content['properties']['parameter']
    except:
        print('Error')
        print(content)
        
    for key in results:
        df[key] = results[key].values()
    df.index = pd.date_range(start=pd.to_datetime(content['header']['start']), 
                         end=pd.to_datetime(content['header']['end']) + pd.DateOffset(hours=23),
                         freq='1H')
    return df

In [ ]:
weather = pd.DataFrame()
for year in range(2014,2022):
    print(f'{year} processing...')
    req = get_weather_from_nasa_power(region_name='Kemerovo', dt_start=f'{year}0101', dt_end=f'{year}1231')
    weather = pd.concat([weather, req])
req = get_weather_from_nasa_power(region_name='Kemerovo', dt_start=f'20220101', dt_end=f'20220901')
weather = pd.concat([weather, req])

weather.to_pickle("DATA\\Weather\\Kemerovo_weather.pkl")

weather

## Скачать календарные признаки

In [6]:
def download_working_days(year_start=2012, year_end=2023):
    hol = holidays.RU()
    index = pd.DatetimeIndex(pd.date_range(start=pd.to_datetime(str(year_start)),
                                           end=(pd.to_datetime(str(year_end)) - pd.Timedelta(hours=1)),
                                           freq='1D'))
    df = pd.DataFrame(columns=['workday', 'weekend', 'shortday', 'holiday'], index=index)
    isdayoffs = ''
    for year in index.year.unique().values:
        print(f'{year} processing...')
        time.sleep(1)
        resp = requests.request('POST', url=f'https://isdayoff.ru/api/getdata?year={year}&pre=1')
        isdayoffs = isdayoffs + resp.text
        
    for i, date in enumerate(index):
        df.at[date, 'holiday'] = int(datetime.date(date.year, date.month, date.day) in hol)
        df.at[date, 'workday'] = 1 if isdayoffs[i] != '1' else 0
        df.at[date, 'weekend'] = 1 if isdayoffs[i] == '1' and datetime.date(date.year, date.month, date.day) not in hol else 0
        df.at[date, 'shortday'] = 1 if isdayoffs[i] == '2' else 0
        
    outfolder = 'DATA/Working_days/'
    os.makedirs(outfolder, exist_ok=True)
    df.to_csv(outfolder + '/working_days.csv', index_label='date')

In [ ]:
download_working_days()

In [7]:
def get_working_days():
    path = os.getcwd() + '\\DATA\\Working_days\\working_days.csv'
    # загрузить сырой файл
    days = pd.read_csv(path)
    # создать щаблон с нужными метками времени
    index = pd.DatetimeIndex(pd.date_range(start=days['date'][0],
                                           end  =days['date'][len(days['date'])-1],
                                           freq ='1H'))
    df = pd.DataFrame(index=index)
    # сделать временной индекс у загруженного массива
    days['date'] = pd.to_datetime(days['date'].astype(str) + ' 00:00:00')
    days.set_index('date', inplace=True)
    # объединить загружнный массив с метками времени
    days = pd.merge(df, days, how='outer', left_index=True, right_index=True)
    days = days.fillna(method='ffill')
    return days

days = get_working_days()
days

,workday,weekend,shortday,holiday
2012-01-01 00:00:00,0.0,0.0,0.0,1.0
2012-01-01 01:00:00,0.0,0.0,0.0,1.0
2012-01-01 02:00:00,0.0,0.0,0.0,1.0
2012-01-01 03:00:00,0.0,0.0,0.0,1.0
2012-01-01 04:00:00,0.0,0.0,0.0,1.0
...,...,...,...,...
2022-12-30 20:00:00,1.0,0.0,0.0,0.0
2022-12-30 21:00:00,1.0,0.0,0.0,0.0
2022-12-30 22:00:00,1.0,0.0,0.0,0.0
2022-12-30 23:00:00,1.0,0.0,0.0,0.0


In [10]:
working_days = pd.read_pickle("DATA\\working_days.pkl")
working_days['month'] = [x.month for x in working_days.index]
working_days['day'] = [x.day for x in working_days.index]
working_days['hour'] = [x.hour for x in working_days.index]
working_days['day_of_week'] = [x.dayofweek for x in working_days.index]
working_days['year'] = [x.year for x in working_days.index]
working_days['index'] = np.arange(0,len(working_days))

working_days.to_pickle("DATA/Working_days/working_days.pkl")

working_days

,workday,weekend,shortday,holiday,month,day,hour,day_of_week,year,index
2012-01-01 00:00:00,0.0,0.0,0.0,1.0,1,1,0,6,2012,0
2012-01-01 01:00:00,0.0,0.0,0.0,1.0,1,1,1,6,2012,1
2012-01-01 02:00:00,0.0,0.0,0.0,1.0,1,1,2,6,2012,2
2012-01-01 03:00:00,0.0,0.0,0.0,1.0,1,1,3,6,2012,3
2012-01-01 04:00:00,0.0,0.0,0.0,1.0,1,1,4,6,2012,4
...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,1.0,0.0,0.0,0.0,12,30,20,4,2022,96404
2022-12-30 21:00:00,1.0,0.0,0.0,0.0,12,30,21,4,2022,96405
2022-12-30 22:00:00,1.0,0.0,0.0,0.0,12,30,22,4,2022,96406
2022-12-30 23:00:00,1.0,0.0,0.0,0.0,12,30,23,4,2022,96407


### Формирование датасетов

In [75]:
ats.columns = ['ats_price']
sobr = sobr[~sobr.index.duplicated(keep='first')].astype('float32')

data = pd.merge(ats, node_prices, how='outer', left_index=True, right_index=True).dropna()
data = pd.merge(data, working_days, how='outer', left_index=True, right_index=True).dropna()
data = pd.merge(data, sobr[['load_planned_sobr', 'generate_planned_sobr']], how='outer', left_index=True, right_index=True).dropna()
classification_data = pd.merge(data, weather[['T2M']], how='outer', left_index=True, right_index=True).dropna().astype('float32')
classification_data = classification_data.loc[:'2022-08-01']
classification_data.to_pickle(f"DATASET\\classification_data.pkl")
classification_data

,ats_price,node_price,workday,weekend,shortday,holiday,month,day,hour,day_of_week,year,index,load_planned_sobr,generate_planned_sobr,T2M
2019-09-15 00:00:00,534.150024,596.798035,0.0,1.0,0.0,0.0,9.0,15.0,0.0,6.0,2019.0,67536.0,3314.781006,1855.880005,6.78
2019-09-15 01:00:00,534.219971,596.328613,0.0,1.0,0.0,0.0,9.0,15.0,1.0,6.0,2019.0,67537.0,3299.569092,2000.879028,7.56
2019-09-15 02:00:00,531.450012,594.732117,0.0,1.0,0.0,0.0,9.0,15.0,2.0,6.0,2019.0,67538.0,3342.320068,2000.879028,8.67
2019-09-15 03:00:00,524.900024,590.206787,0.0,1.0,0.0,0.0,9.0,15.0,3.0,6.0,2019.0,67539.0,3368.802002,1855.880005,9.66
2019-09-15 04:00:00,658.750000,721.414429,0.0,1.0,0.0,0.0,9.0,15.0,4.0,6.0,2019.0,67540.0,3534.553955,1886.124023,10.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01 19:00:00,1264.670044,1307.668457,1.0,0.0,0.0,0.0,8.0,1.0,19.0,0.0,2022.0,92779.0,3144.154053,2140.100098,14.07
2022-08-01 20:00:00,1240.239990,1285.128174,1.0,0.0,0.0,0.0,8.0,1.0,20.0,0.0,2022.0,92780.0,3053.440918,2140.100098,13.66
2022-08-01 21:00:00,1139.569946,1182.494141,1.0,0.0,0.0,0.0,8.0,1.0,21.0,0.0,2022.0,92781.0,3027.010010,2150.100098,13.30
2022-08-01 22:00:00,1048.040039,1084.094604,1.0,0.0,0.0,0.0,8.0,1.0,22.0,0.0,2022.0,92782.0,3027.762939,1847.401001,13.08


In [72]:
data = pd.merge(ats, working_days, how='outer', left_index=True, right_index=True).dropna()
data = pd.merge(data, sobr[['load_planned_sobr', 'generate_planned_sobr']], how='outer', left_index=True, right_index=True).dropna()
experiment_rsv = pd.merge(data, weather[['T2M']], how='outer', left_index=True, right_index=True).dropna().astype('float32')
experiment_rsv = experiment_rsv.loc[:'2022-07-31']
experiment_rsv.to_pickle(f"DATASET\\experiment_rsv.pkl")
experiment_rsv

,ats_price,workday,weekend,shortday,holiday,month,day,hour,day_of_week,year,index,load_planned_sobr,generate_planned_sobr,T2M
2014-01-01 00:00:00,729.109985,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,2014.0,17544.0,3429.000000,2255.000000,-10.000000
2014-01-01 01:00:00,716.330017,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2014.0,17545.0,3462.000000,2255.000000,-10.020000
2014-01-01 02:00:00,709.099976,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,2014.0,17546.0,3550.000000,2255.000000,-10.020000
2014-01-01 03:00:00,702.099976,0.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,2014.0,17547.0,3539.000000,2255.000000,-10.140000
2014-01-01 04:00:00,710.919983,0.0,0.0,0.0,1.0,1.0,1.0,4.0,2.0,2014.0,17548.0,3518.000000,2255.000000,-10.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-31 19:00:00,1511.290039,0.0,1.0,0.0,0.0,7.0,31.0,19.0,6.0,2022.0,92755.0,3164.051025,2015.900024,17.730000
2022-07-31 20:00:00,1493.819946,0.0,1.0,0.0,0.0,7.0,31.0,20.0,6.0,2022.0,92756.0,3087.885010,2025.900024,17.280001
2022-07-31 21:00:00,1396.079956,0.0,1.0,0.0,0.0,7.0,31.0,21.0,6.0,2022.0,92757.0,3038.926025,1995.900024,16.889999
2022-07-31 22:00:00,1259.859985,0.0,1.0,0.0,0.0,7.0,31.0,22.0,6.0,2022.0,92758.0,3041.237061,1995.900024,16.690001
